#### esquemas


In [453]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyodbc
import re

#### MODELO 

In [ ]:
import json
import re
import os
import pyodbc
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Detalles de Conexión a la Base de Datos ---
server = ''
database = ''
username = ''
password = ''

# Construir la cadena de conexión
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Modelo de Lenguaje Grande) ---
os.environ["GOOGLE_API_KEY"] = "" 
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-002")

#### META DATOS

In [ ]:
#- DEFINICIÓN DE TABLAS Y METADATOS A INCLUIR ---
# 1. Lista de las tablas EXACTAS que quieres procesar
TABLES_TO_INCLUDE = ['PART_MASTER', 'ORDER_MASTER','Part_Master_Ext', 'Vendor_Master','_Categorias','Transaction_History']

# 2. Diccionario de metadatos enriquecidos para estas tablas y sus columnas clave

METADATOS_DB = {
    "PART_MASTER": {
        "friendly_name": "Maestro de Partes/Productos",
        "description": "Contiene información detallada sobre las partes o productos disponibles.",
        "columns": {
            "PRTNUM_01": {"friendly_name": "ID de Parte", "description": "Identificador único de la parte/producto."},
            "PMDES1_01": {"friendly_name": "Descripción de Parte", "description": "Descripción detallada de la parte/producto."},
            "ON_HAND_QTY": {"friendly_name": "Cantidad en Stock", "description": "Cantidad actual de unidades disponibles en inventario."},
            "UOM": {"friendly_name": "Unidad de Medida", "description": "Unidad en la que se mide la parte (ej. PZA, KG)."}
           
        }
    },
    "ORDER_MASTER": {
        "friendly_name": "Maestro de Órdenes",
        "description": "Registra los detalles principales de ordenes de produccion, de compra de materia prima.",
        "columns": {
            "ORDNUM_10": {"friendly_name": "ID de Orden", "description": "Identificador de cada orden"},
            "PRTNUM_10": {"friendly_name": "ID de producto", "description": "Identificador de la parte/producto asociado a la orden."},
            "CURDUE_10": {"friendly_name": "Fecha de Orden", "description": "Fecha en que se realizó la orden."},
            "DUEQTY_10": {"friendly_name": "cantidad", "description": "cantidad de la orden."},
            "COST_10": {"friendly_name": "costo", "description": "costo orden."},
            "STATUS_10": {"friendly_name": "Estado de Orden", "description": "Estado actual de la orden (ej. 'PENDIENTE', 'COMPLETADO', 'CANCELADO')."},
            "VENID_10": {"friendly_name": "ID proveedor", "description": "Identificador proveedor."},
            "UDFKEY_10": {"friendly_name": "ID del contrato", "description": "Identificador único que relaciona la orden de produccion de materia prima con el contrato del cliente."},
            "LINNUM_10": {"friendly_name": "Número de línea", "description": "Identificador que representa cada línea individual dentro de una orden o pedido."},
            "LINNUM_10": {"friendly_name": "Número de entrega", "description": "Identificador que especifica cada una de las entregas."},
            "TYPE_10": {"friendly_name": "tipo de orden ", "description": "Clasifica la naturaleza o categoría específica de la orden en el sistema. Los valores posibles incluyen: CU, FC, MF, MS, NI, PL, PO, PR, SO, SR, entre otros."},
            "ORDER_10": {"friendly_name": "ID LARGO para usar cono join con la tabla Transaction_History", "description": "Identificador se relaciona directamente con la tabla 'Transaction_History'."},
            
            
        }
    },
    "Part_Master_Ext": {
        "friendly_name": "Maestro de Partes Extendido",
        "description": "Contiene información extendida sobre las partes o productos, pero con informacion adicional al part_master.",
        "columns": {
            "PRTNUM_01": {"friendly_name": "ID de Parte extendido", "description": "Identificador único de la parte/producto."},
            "Categoría": {"friendly_name": "ID Categoría producto", "description": "ID del producto materia prima."}
        }
    },
    "Vendor_Master": {
        "friendly_name": "tabla información de proveedores, donde esta el nombredel proveedor",
        "description": "Contiene información de los proveedores, Descripcion del proveedor y incluyendo detalles de contacto y ubicación.",
        "columns": {
            "VENID_08": {"friendly_name": "ID del proveedor", "description": "Identificador único del proveedor."},
            "COMNAM_08": {"friendly_name": "descripcion del proveedor", "description": "descripcion del nombre completo  del social del proveedor ."},
            "CNTRY_08": {"friendly_name": "País del Proveedor", "description": "país de origen donde el proveedor."},
        }
    
},
"Celsa_Categorias": {
        "friendly_name": "Tabla de Categorías de Materia Prima",
        "description": "Contiene la clasificación y descripción de las diferentes categorías de materia prima utilizadas por Celsa.",
        "columns": {
            "Id": {"friendly_name": "ID de Categoría", "description": "Identificador único de la categoría de materia prima."},
            "categoria": {"friendly_name": "descripcion de la Categoría de Materia Prima", "description": "Contiene la clasificación y detalles de las categorías de materia prima."}
        }
    
},
"Transaction_History": {
        "friendly_name": "Historial General de Transacciones de Inventario y Movimientos",
        "description": "Contiene un registro exhaustivo de todas las transacciones que afectan el inventario y los movimientos de bienes de la empresa. Esto incluye ingresos (ej. compras, recepciones), salidas (ej. ventas, despachos a clientes), transferencias internas entre bodegas, y consumos/despachos de materia prima a producción. Cada transacción detalla el tipo de movimiento, los artículos involucrados y las cantidades.",
        "columns": {
            "ORDNUM_15": {"friendly_name": "Número de Orden/Transacción", "description": "dentificador único de la orden o transacción específica que originó el movimiento. Puede referirse a una orden de compra, de venta, un número de transferencia interna, o cualquier otro identificador de la operación."},
            "TNXDTE_15": {"friendly_name": "Fecha de Movimiento/Transacción", "description": "Indica la fecha  exacta en que se realizó o registró un movimiento o transacción en el inventario o sistema."},
            "TNXQTY_15": {"friendly_name": "Cantidad de Transacción", "description": "Campo que registra la cantidad de artículos o unidades involucradas en la transacción o movimiento."},
            "TNXCDE_15": {"friendly_name": "Tipo de Transacción/Movimiento", "description": "Indica el tipo de acción o movimiento específico que se realiza dentro de una transacción. Valores comunes incluyen: 'R' (Ingresos/Recepciones), 'I' (Salidas/Despachos), y otros códigos como A, C, E, F, N, P, Q, S, T, U, V, W, X, Y, Z, cada uno representando una acción distinta en el sistema."}
            
        }
   
}
   
    
}

#### TEMPLATE

In [456]:
template = """
--- Plantilla para la Generación de SQL ---
Eres un generador de SQL experto. Tu tarea es traducir preguntas en lenguaje natural a consultas SQL precisas para **SQL Server de Microsoft**, basándote en el esquema de base de datos proporcionado.

---

**Instrucciones:**
1.  El `Esquema` proporcionado incluye el nombre técnico de la tabla, un `_friendly_name` (nombre amigable) y una `_description` (descripción) para cada tabla, además de las columnas. Cada columna también tiene su `friendly_name` y `description`.
2.  Analiza cuidadosamente el `Esquema` (usa las descripciones y nombres amigables para entenderlo mejor) para determinar las tablas y columnas relevantes.
3.  Interpreta la `Pregunta del usuario` para determinar los datos exactos que se necesitan y las operaciones requeridas (ej. SELECT, JOIN, WHERE, GROUP BY, ORDER BY).
4.  **Genera *solamente* la consulta SQL.** Utiliza los nombres técnicos EXACTOS de las tablas y columnas de la base de datos (los que aparecen como claves en el esquema, no los "friendly_name") en la consulta SQL. No incluyas explicaciones, comentarios ni texto adicional.
5.  Asegúrate de que la consulta SQL sea sintácticamente correcta y que cumpla con las prácticas SQL estándar.
6.  Instrucción de Fechas: Para cualquier filtro que involucre fechas, asegúrate de castear la columna de la tabla al tipo `DATE` y luego comparar con una fecha casteada explícitamente como `CAST('YYYY-MM-DD' AS DATE)`.
7.  Uso de Joins: Cuando la pregunta del usuario implique buscar datos relacionados entre dos o más tablas, utiliza **`INNER JOIN`** para combinar las tablas en función de sus columnas relacionadas (claves).
8.  Restricción de Operadores de Conjunto:NO uses el operador `INTERSECT` bajo ninguna circunstancia para combinar o comparar conjuntos de resultados.

**Esquema:**
{schema}

**Pregunta del usuario:**
{query}

**Salida (solo SQL):**
"""



#### EXRTRAER ESQUEMA DB

In [ ]:

def extract_schema_pyodbc(connection_string, tables_to_include, metadatos_db):
    schema_for_llm = {}
    cnxn = None
    try:
        cnxn = pyodbc.connect(connection_string)
        cursor = cnxn.cursor()

        
        for table_name in tables_to_include:
            if table_name in metadatos_db: 
                table_meta = metadatos_db[table_name]
                llm_table_entry = {
                    "_friendly_name": table_meta["friendly_name"],
                    "_description": table_meta["description"],
                    "columns": {}
                }

                # Obtener las columnas reales de la tabla de la BD
                cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{table_name}'")
                db_columns = {col[0]: col[1] for col in cursor.fetchall()}

                for col_name, col_type in db_columns.items():
                    if col_name in table_meta["columns"]: # Usar metadatos para las columnas mapeadas
                        col_meta = table_meta["columns"][col_name]
                        llm_table_entry["columns"][col_name] = {
                            "friendly_name": col_meta["friendly_name"],
                            "description": col_meta["description"],
                            "type": col_type
                        }
                    else:
                        
                        llm_table_entry["columns"][col_name] = {
                            "friendly_name": col_name,
                            "description": "Sin descripción específica (nombre técnico).",
                            "type": col_type
                        }
                schema_for_llm[table_name] = llm_table_entry
                print(f"  - Esquema enriquecido y filtrado para la tabla: {table_name}")
            else:
                print(f"ADVERTENCIA: La tabla '{table_name}' está en TABLES_TO_INCLUDE pero no tiene metadatos en METADATOS_DB. Será omitida o procesada sin enriquecimiento.")
                

    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"Error de base de datos: {sqlstate}")
        print(ex)
        return None
    finally:
        if cnxn:
            cnxn.close()

    return json.dumps(schema_for_llm, indent=4)


####------- LANGCHAIN ---------#####

# --- Función para Generar Consultas SQL ---
def to_sql_query(query, schema, llm_model):
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm_model
    response = chain.invoke({"query": query, "schema": schema})
    if hasattr(response, 'content'):
        return clean_text(response.content)
    else:
        return clean_text(str(response))
    






# --- Utilidad para Limpiar Texto ---
import re

import re


def clean_text(text: str) -> str:
    # Elimina los bloques de pensamiento
    cleaned_text = re.sub(r"<.*?think.*?>.*?</.*?think.*?>", "", text, flags=re.DOTALL)

    # Decodifica secuencias Unicode como '\u00eda' a su carácter real ('ía')
    
    try:
        
        cleaned_text = cleaned_text.encode('latin1').decode('unicode_escape')
    except (UnicodeDecodeError, UnicodeEncodeError):
        
        pass 

    # Elimina el envoltorio de bloques de código Markdown (```sql...```)
    
    if cleaned_text.strip().startswith('```sql') and cleaned_text.strip().endswith('```'):
        # Elimina '```sql' del inicio y '```' del final
        cleaned_text = cleaned_text.strip()[len('```sql'):-len('```')]
       
        cleaned_text = cleaned_text.strip()

    return cleaned_text.strip()



--- Iniciando Extracción de Esquema Filtrado ---


##### log

In [458]:
schema = extract_schema_pyodbc(connection_string, TABLES_TO_INCLUDE, METADATOS_DB)

  - Esquema enriquecido y filtrado para la tabla: PART_MASTER
  - Esquema enriquecido y filtrado para la tabla: ORDER_MASTER
  - Esquema enriquecido y filtrado para la tabla: Part_Master_Ext
  - Esquema enriquecido y filtrado para la tabla: Vendor_Master
  - Esquema enriquecido y filtrado para la tabla: Celsa_Categorias
  - Esquema enriquecido y filtrado para la tabla: Transaction_History


#### PREGUNTA 

In [459]:
test_queries = [
    """
    traeme del maestro de ordenes las ordenes en estado 4 y 5 y tipo PO, SO
    y con fecha año 2025 y mes mayo, y con estos datos cruzarlos con los ingresos
    y ademas dame la descripcion del producto y tambien el codigo del producto
    
    """
]

#### CORRER MODELO

In [460]:
if schema is None:
    print("\nERROR: No se pudo recuperar el esquema de las tablas especificadas. Verifique los detalles de conexión, nombres de tablas y disponibilidad de la base de datos.")
else:
    #print("\n--- Esquema Extraído Exitosamente para Tablas Seleccionadas ---")
    #print("Esquema:")
    #print(schema)

    # --- Consultas de Prueba enfocadas solo en PART_MASTER y ORDER_MASTER ---
    test_queries ,

    #print("\n--- Generando Consultas SQL para las tablas seleccionadas ---")
    for i, user_query in enumerate(test_queries):
        #print(f"\n--- Consulta de Prueba {i+1} ---")
        #print(f"Pregunta del Usuario: {user_query}")
        try:
            sql_output = to_sql_query(user_query, schema, llm)
            #print("Consulta SQL Generada:")
            #print(sql_output)
        except Exception as e:
            print(f"Ocurrió un error al generar la consulta SQL: {e}")

#### CONSULTA

In [461]:
print(sql_output)

SELECT
	OM.ORDNUM_10,
	OM.STATUS_10,
	OM.TYPE_10,
	OM.CURDUE_10,
	PM.PMDES1_01,
	PM.PRTNUM_01
FROM
	ORDER_MASTER AS OM
INNER JOIN
	PART_MASTER AS PM ON OM.PRTNUM_10 = PM.PRTNUM_01
INNER JOIN
	Transaction_History AS TH ON OM.ORDER_10 = TH.ORDNUM_15
WHERE
	OM.STATUS_10 IN ('4', '5')
	AND OM.TYPE_10 IN ('PO', 'SO')
	AND CAST(OM.CURDUE_10 AS DATE) >= CAST('2025-05-01' AS DATE)
	AND CAST(OM.CURDUE_10 AS DATE) <= CAST('2025-05-31' AS DATE)
	AND TH.TNXCDE_15 = 'R'


#### EJECUTAR CONSULTA

In [462]:
import pyodbc

def execute_sql_query(sql_query: str, connection_string: str):

    conn = None
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        cursor.execute(sql_query)

        # Si es una consulta SELECT, recuperamos los resultados
        if sql_query.strip().upper().startswith("SELECT"):
            columns = [column[0] for column in cursor.description] # Nombres de las columnas
            results = []
            for row in cursor.fetchall():
                results.append(dict(zip(columns, row))) # Convertimos cada fila en un diccionario
            return pd.DataFrame(results)
        else:
            # Para INSERT, UPDATE, DELETE, etc., confirmamos los cambios
            conn.commit()
            return "Comando ejecutado exitosamente."
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        return f"Error de base de datos ({sqlstate}): {ex}"
    finally:
        # Nos aseguramos de cerrar la conexión
        if conn:
            conn.close()



In [463]:
resultado_de_la_db = execute_sql_query(sql_output, connection_string)

#### RESULTADO

In [464]:
resultado_de_la_db

,ORDNUM_10,STATUS_10,TYPE_10,CURDUE_10,PMDES1_01,PRTNUM_01
0,77727828,4,PO,2025-05-12,EMPAQ FOTOC.NALSIGMA11213 ...,11136
1,77727833,4,PO,2025-05-09,SOPORTE PCBS CON BAYONETA H=3.2mm ...,77846
2,77727835,4,PO,2025-05-12,CABLE ENCAUCHETADO 3*18 ...,18985
3,77727835,4,PO,2025-05-06,TERMOENCOGIBLE 3MM NEGRO ...,76016
4,77727843,4,PO,2025-05-09,BOLSA BURBUJA PEQ. PROTEC ...,65547
...,...,...,...,...,...,...
588,77727804,4,PO,2025-05-23,PSP -01 INADISA ...,16722
589,77727804,4,PO,2025-05-23,PSP -01 INADISA ...,16722
590,77727807,4,PO,2025-05-15,CAJA CARTON CCIX PEQ RECICLAJE ...,16761
591,77727808,4,PO,2025-05-15,ARANDELA ESTRIADA 5/32 INOX ...,13236
